In [1]:
from pathlib import Path
import pickle
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Nadam
from keras.callbacks import ModelCheckpoint, TensorBoard, LearningRateScheduler
from models.models import *
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


In [2]:
# Number of searches performed
hyperparameter_iterations = 5
# epochs*(1frozen + 2unfrozen)
epochs = 15
# Batch size too small, bad approximation of the global loss.
# Too large, gets stuck in minima.
batch_size = 16

# Default input size for many architectures
image_size = (224, 224)

In [3]:
# Dataset folder, expects three subdirectories:
# validation, train, test
dataset = Path('Adience_dataset/')
val_test_gen = ImageDataGenerator(rescale=1./255)
validation = val_test_gen.flow_from_directory(dataset / 'validation',
                                              target_size=image_size,
                                              batch_size=batch_size,
                                              class_mode='categorical', shuffle=False)

test = val_test_gen.flow_from_directory(dataset / 'test',
                                        target_size=image_size,
                                        batch_size=1,
                                        class_mode='categorical', shuffle=False)
validation_steps = int(np.ceil(validation.n/batch_size))

Found 790 images belonging to 2 classes.
Found 396 images belonging to 2 classes.


In [4]:
# Random search
# Architecture
#architecture = MobileNetGenderFConnected(image_size, alpha=0.75, filter_factor=398, l2_factor=6.45E-04)
#architecture = MobileNetGender(image_size, alpha, filter_factor, l2_factor)
#architecture = InceptionGenderV3(image_size, filter_factor=447, l2_factor=7.50E-04)
architecture = VGGGender(image_size, filter_factor=585, l2_factor=1.05E-04)

train_gen = ImageDataGenerator(rescale=1./255,
                               horizontal_flip=True,
                               shear_range=3.22E+02,
                               rotation_range=155)

# Load weights
model = architecture.model


In [5]:
#model.load_weights('Saved models/InceptionV3_447_l2_7.50E-04_rotation_155_shear_3.22E+02.hdf5')
model.load_weights('Saved models/VGG16_585_l2_1.05E-04_rotation_136_shear_1.89E+02.hdf5')

In [6]:
model_save_name = 'transfer_'
model_save_name += architecture.name
log_output = os.path.join('logs', model_save_name)
model_save_name += '.hdf5'
os.makedirs(log_output, exist_ok=True)
tb = TensorBoard(log_output)
mc = ModelCheckpoint(os.path.join(
     'Saved models transfer', model_save_name), save_best_only=True, save_weights_only=True)

#mc = ModelCheckpoint(os.path.join(
#    'Saved models_adience', model_save_name), save_best_only=True, save_weights_only=True)

callbacks = [tb, mc]

train = train_gen.flow_from_directory(dataset / 'train',
                                      target_size=image_size,
                                      batch_size=batch_size,
                                      class_mode='categorical')
total = train.n
males = np.count_nonzero(train.classes)
females = total - males
# If males outnumber females, we get higher loss for females
weights = {0: males/females, 1: 1}

steps_per_epoch = int(np.ceil(total/batch_size))


# Learning rate is low so that we don't ruin our parameters
model.compile(optimizer=Nadam(lr=2e-6),  # default is 2e-3
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_frozen = model.fit_generator(train,
                                     epochs=15,
                                     steps_per_epoch=steps_per_epoch,
                                     validation_data=validation,
                                     validation_steps=validation_steps,
                                     class_weight=weights,
                                     callbacks=callbacks)

# Unfreezing all layers
for layer in model.layers:
    layer.trainable = True

# Learning rate is low so that we don't ruin our parameters
model.compile(optimizer=Nadam(lr=2e-6),  # default is 2e-3
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_unfrozen = model.fit_generator(train,
                    epochs=40,
                   steps_per_epoch=steps_per_epoch,
                   validation_data=validation,
                   validation_steps=validation_steps,
                   class_weight=weights,
                   callbacks=callbacks)

test_loss, test_acc = model.evaluate_generator(
    test, steps=test.n, verbose=1)
with open(os.path.join(log_output, 'histories'), 'wb') as f:
    pickle.dump([history_frozen.history,
                 history_unfrozen.history, test_loss, test_acc], f)
print(f'{model_save_name}\nreached an accuracy of {test_acc:.3f} and loss of {test_loss:.3f}')

Found 6700 images belonging to 2 classes.
Epoch 1/15
419/419 [==============================] - 134s 319ms/step - loss: 1.4687 - acc: 0.5475 - val_loss: 0.8389 - val_acc: 0.7430
Epoch 2/15
419/419 [==============================] - 127s 303ms/step - loss: 1.4867 - acc: 0.5357 - val_loss: 0.8305 - val_acc: 0.7456
Epoch 3/15
419/419 [==============================] - 127s 303ms/step - loss: 1.4659 - acc: 0.5429 - val_loss: 0.8182 - val_acc: 0.7456
Epoch 4/15
419/419 [==============================] - 128s 306ms/step - loss: 1.4526 - acc: 0.5402 - val_loss: 0.7952 - val_acc: 0.7494
Epoch 5/15
419/419 [==============================] - 127s 304ms/step - loss: 1.4097 - acc: 0.5524 - val_loss: 0.7785 - val_acc: 0.7570
Epoch 6/15
419/419 [==============================] - 128s 305ms/step - loss: 1.3781 - acc: 0.5591 - val_loss: 0.7745 - val_acc: 0.7557
Epoch 7/15
419/419 [==============================] - 127s 303ms/step - loss: 1.3879 - acc: 0.5471 - val_loss: 0.7884 - val_acc: 0.7532
Epoch 